In [12]:
from fly import Fly, OK, WON, LOST
import matplotlib.pyplot as plt
import random
import networkx as nx
from graphviz import Digraph
from IPython.display import display, Image
from itertools import product
import copy
from tqdm import tqdm


In [3]:
# Générer tous les états possibles
all_possible_boards = set(product(range(6), repeat=4))

print(f"Total possible boards: {len(all_possible_boards)}")

Total possible boards: 1296


In [4]:
next_boards_dict = {}

for board in all_possible_boards:
    fly = Fly(board)
    next_boards = []
    if fly.state == OK:
        for move in range(4):
            try:
                next_board, next_state = tuple(fly.explore(move))
                next_boards.append({'move': move, 'board': next_board, 'state': next_state})
            except ValueError as e:
                if str(e) == "Game is already over":
                    continue
                else:
                    raise
    next_boards_dict[board] = next_boards

next_boards_dict

{(4, 2, 0, 5): [],
 (2, 3, 1, 4): [{'move': 0, 'board': [0, 0, 1, 4], 'state': 1},
  {'move': 1, 'board': [2, 4, 1, 4], 'state': 1},
  {'move': 2, 'board': [2, 3, 2, 4], 'state': 1},
  {'move': 3, 'board': [2, 3, 1, 5], 'state': 3}],
 (3, 3, 3, 0): [{'move': 0, 'board': [4, 3, 3, 0], 'state': 1},
  {'move': 1, 'board': [3, 4, 3, 0], 'state': 1},
  {'move': 2, 'board': [3, 3, 4, 0], 'state': 1},
  {'move': 3, 'board': [3, 3, 3, 1], 'state': 1}],
 (2, 5, 5, 3): [],
 (0, 5, 3, 2): [],
 (4, 5, 0, 2): [],
 (5, 0, 0, 5): [],
 (5, 3, 5, 0): [],
 (1, 2, 5, 0): [],
 (2, 4, 2, 1): [{'move': 0, 'board': [3, 4, 2, 1], 'state': 1},
  {'move': 1, 'board': [2, 5, 2, 1], 'state': 3},
  {'move': 2, 'board': [2, 4, 3, 1], 'state': 1},
  {'move': 3, 'board': [2, 4, 0, 0], 'state': 1}],
 (3, 4, 3, 2): [{'move': 0, 'board': [0, 0, 3, 2], 'state': 1},
  {'move': 1, 'board': [3, 5, 3, 2], 'state': 3},
  {'move': 2, 'board': [3, 0, 0, 2], 'state': 1},
  {'move': 3, 'board': [3, 4, 0, 0], 'state': 1}],
 (5, 5,

In [5]:
nodes_dict = {}

# Initialize nodes_dict with 'prev' and 'next' lists
for board in next_boards_dict:
    fly = Fly(board)
    nodes_dict[board] = {'prev': [], 'next': next_boards_dict[board], 'state': fly.state}

# Populate the 'prev' lists
for board, next_boards in next_boards_dict.items():
    for data in next_boards:
        move, next_board, state = data.values()
        next_board = tuple(next_board)
        if next_board in nodes_dict:
            # nodes_dict[next_board]['prev'].append((move, board, state))
            nodes_dict[next_board]['prev'].append({'move': move, 'board': board, 'state': state})

nodes_dict

{(4, 2, 0, 5): {'prev': [{'move': 3, 'board': (4, 2, 0, 4), 'state': 3}],
  'next': [],
  'state': 3},
 (2,
  3,
  1,
  4): {'prev': [{'move': 1, 'board': (2, 2, 1, 4), 'state': 1},
   {'move': 0, 'board': (1, 3, 1, 4), 'state': 1},
   {'move': 3, 'board': (2, 3, 1, 3), 'state': 1},
   {'move': 2, 'board': (2, 3, 0, 4), 'state': 1}], 'next': [{'move': 0,
    'board': [0, 0, 1, 4],
    'state': 1},
   {'move': 1, 'board': [2, 4, 1, 4], 'state': 1},
   {'move': 2, 'board': [2, 3, 2, 4], 'state': 1},
   {'move': 3, 'board': [2, 3, 1, 5], 'state': 3}], 'state': 1},
 (3, 3, 3, 0): {'prev': [],
  'next': [{'move': 0, 'board': [4, 3, 3, 0], 'state': 1},
   {'move': 1, 'board': [3, 4, 3, 0], 'state': 1},
   {'move': 2, 'board': [3, 3, 4, 0], 'state': 1},
   {'move': 3, 'board': [3, 3, 3, 1], 'state': 1}],
  'state': 1},
 (2, 5, 5, 3): {'prev': [], 'next': [], 'state': 3},
 (0, 5, 3, 2): {'prev': [{'move': 1, 'board': (0, 4, 3, 2), 'state': 3}],
  'next': [],
  'state': 3},
 (4, 5, 0, 2): {'pre

In [6]:
def find_predecessors(node, nodes_dict):
    predecessors = {}

    def recurse(current_node):
        for predecessor in nodes_dict[current_node]['prev']:
            pred_board = tuple(predecessor['board'])
            if pred_board not in predecessors:
                predecessors[pred_board] = {'move': predecessor['move'], 'state': predecessor['state']}
                recurse(pred_board)

    recurse(node)
    return predecessors


In [7]:
def find_predecessors_bfs(node, nodes_dict):
    predecessors = {}
    queue = [(node, 0)]

    while queue:
        current_node, depth = queue.pop(0)
        for predecessor in nodes_dict[current_node]['prev']:
            pred_board = tuple(predecessor['board'])
            if pred_board not in predecessors:
                predecessors[pred_board] = {'move': predecessor['move'], 'state': predecessor['state'], 'depth': depth}
                queue.append((pred_board, depth + 1))
                predecessor['state'] = 5 +depth

    return predecessors


In [8]:
target_node = (0, 0, 0, 0)
nodes_dict_copy = copy.deepcopy(nodes_dict)

predecessors = find_predecessors_bfs(target_node, nodes_dict_copy)
predecessors

{(0, 4, 3, 0): {'move': 2, 'state': 2, 'depth': 0},
 (0, 1, 0, 0): {'move': 0, 'state': 2, 'depth': 0},
 (4, 3, 0, 0): {'move': 1, 'state': 2, 'depth': 0},
 (3, 4, 0, 0): {'move': 0, 'state': 2, 'depth': 0},
 (1, 2, 0, 0): {'move': 0, 'state': 2, 'depth': 0},
 (2, 3, 0, 0): {'move': 0, 'state': 2, 'depth': 0},
 (3, 2, 0, 0): {'move': 1, 'state': 2, 'depth': 0},
 (0, 3, 4, 0): {'move': 1, 'state': 2, 'depth': 0},
 (0, 1, 2, 0): {'move': 1, 'state': 2, 'depth': 0},
 (0, 3, 2, 0): {'move': 2, 'state': 2, 'depth': 0},
 (0, 2, 1, 0): {'move': 2, 'state': 2, 'depth': 0},
 (0, 2, 3, 0): {'move': 1, 'state': 2, 'depth': 0},
 (0, 0, 1, 0): {'move': 1, 'state': 2, 'depth': 0},
 (0, 0, 3, 2): {'move': 3, 'state': 2, 'depth': 0},
 (1, 0, 0, 0): {'move': 1, 'state': 2, 'depth': 0},
 (0, 0, 1, 2): {'move': 2, 'state': 2, 'depth': 0},
 (2, 1, 0, 0): {'move': 1, 'state': 2, 'depth': 0},
 (0, 0, 3, 4): {'move': 2, 'state': 2, 'depth': 0},
 (0, 0, 2, 1): {'move': 3, 'state': 2, 'depth': 0},
 (0, 0, 0, 1

In [9]:
len(nodes_dict_copy)

1296

In [13]:

def visualize_nodes_dict_copy(nodes_dict_copy):
    dot = Digraph()

    # Add nodes with colors based on their state
    for board, data in tqdm(nodes_dict_copy.items(), desc="Adding nodes"):
        state = data['state']
        color = 'yellow' if state == OK else 'red' if state == LOST else 'blue'
        dot.node(str(board), str(board), style='filled', fillcolor=color)

    # Add edges with colors based on the depth
    for board, data in tqdm(nodes_dict_copy.items(), desc="Adding edges"):
        for prev in data['prev']:
            prev_board = tuple(prev['board'])
            depth = prev['state'] - 5 if 'depth' in prev else 0
            edge_color = 'blue' if depth > 4 else 'black'
            dot.edge(str(prev_board), str(board), color=edge_color)

    return dot


In [ ]:
graph = visualize_nodes_dict_copy(nodes_dict_copy)
graph

Adding edges: 100%|██████████| 1296/1296 [00:00<00:00, 15999.65it/s]


In [ ]:
def build_graph(chemins):
    dot = Digraph()
    for path, result in chemins:
        for i in range(len(path) - 1):
            move, board = path[i]
            next_move, next_board = path[i + 1]
            dot.node(str(board), str(board))
            dot.node(str(next_board), str(next_board))
            dot.edge(str(board), str(next_board), label=str(move))
    return dot


def visualize_graph(graph, chemins):
    graph.attr("node", shape="circle")
    for path, result in chemins:
        color = "blue" if result == "won" else "red"
        for move, board in path:
            graph.node(str(board), style="filled", fillcolor=color)
    graph.view()

In [ ]:

# Faire une copie de nodes_dict
nodes_dict_copy = copy.deepcopy(nodes_dict)

# Trouver les nœuds gagnants
winning_nodes = [board for board, data in nodes_dict_copy.items() if data['state'] == WON]

# Fonction pour propager l'état gagnant aux nœuds prédécesseurs
def propagate_winning_state(board):
    for move, prev_board, state in nodes_dict_copy[board]['prev']:
        if nodes_dict_copy[prev_board]['state'] != WON:
            nodes_dict_copy[prev_board]['state'] = WON
            propagate_winning_state(prev_board)

# Propager l'état gagnant à partir des nœuds gagnants
for board in winning_nodes:
    propagate_winning_state(board)

# Mettre à jour l'état des successeurs dans les tuples (move, board, state)
for board, data in nodes_dict_copy.items():
    if data['state'] == WON:
        for i, (move, next_board, state) in enumerate(data['next']):
            if nodes_dict_copy[tuple(next_board)]['state'] == WON:
                data['next'][i] = (move, next_board, WON)

nodes_dict_copy

In [ ]:
path_dict = {}

# Initialize path_dict with the state of each node
for board, data in nodes_dict.items():
    path_dict[board] = data['state']

# Propagate the winning state to predecessor nodes
def propagate_winning_state(board):
    if path_dict[board] == WON:
        for move, prev_board, state in nodes_dict[board]['prev']:
            if path_dict[prev_board] != WON:
                path_dict[prev_board] = WON
                propagate_winning_state(prev_board)

# Start propagation from winning boards
for board, data in nodes_dict.items():
    if data['state'] == WON:
        propagate_winning_state(board)

# Update the state of the successors in the tuples (move, board, state)
for board, data in nodes_dict.items():
    if path_dict[board] == WON:
        for i, (move, next_board, state) in enumerate(data['next']):
            if path_dict[tuple(next_board)] == WON:
                data['next'][i] = (move, next_board, WON)

path_dict

In [ ]:
nodes_dict[(2, 3, 1, 4)]

In [37]:
def brute_force(max_paths: int, max_depth: int, verbose: bool = False) -> dict:
    chemins = {'WON': [], 'LOST': []}

    def dfs(fly: Fly, path: list, depth: int) -> bool:
        if len(chemins['WON']) + len(chemins['LOST']) >= max_paths or depth > max_depth:
            return False
        state = fly.get_state()
        if state != OK:
            if state in (WON, LOST):
                path_result = "WON" if state == WON else "LOST"
                chemins[path_result].append(path[:])
                if verbose:
                    print(f"Path when {path_result}: {[move[0] for move in path]}")
            return state == WON

        for i in range(4):
            new_fly = Fly(verbose=False)
            new_fly.board = fly.board[:]
            new_fly.state = fly.state
            new_fly.play(i)
            path.append((i, new_fly.board[:]))
            dfs(new_fly, path, depth + 1)
            path.pop()
        return False

    fly = Fly(verbose=verbose)
    initial_state = fly.board[:]
    dfs(fly, [(None, initial_state)], 0)

    if len(chemins['WON']) > 0:
        print("Found a winning sequence within the path limit.")
    else:
        print("No winning sequence found within the path limit.")

    return chemins


In [44]:
def build_q_dict(paths):
    q_dict = {}
    learning_rate = 0.1
    discount_factor = 0.9

    for result, path_list in paths.items():
        reward = 1 if result == "WON" else -1
        for path in path_list:
            for i in range(len(path) - 1):
                _, board = path[i]
                move, next_board = path[i + 1]
                board_tuple = tuple(board)
                next_board_tuple = tuple(next_board)
                if board_tuple not in q_dict:
                    q_dict[board_tuple] = {}
                if move not in q_dict[board_tuple]:
                    q_dict[board_tuple][move] = (0, next_board_tuple)
                q_value, _ = q_dict[board_tuple][move]
                next_q_value = max(q_dict[next_board_tuple].values(), key=lambda x: x[0])[0] if next_board_tuple in q_dict else 0
                q_dict[board_tuple][move] = (q_value + learning_rate * (reward + discount_factor * next_q_value - q_value), next_board_tuple)

    return q_dict

def visualize_q_dict(q_dict):
    dot = Digraph()
    for board, moves in q_dict.items():
        for move, (q_value, next_board_tuple) in moves.items():
            color = "blue" if q_value > 0 else "red"
            dot.node(str(board), str(board))
            dot.node(str(next_board_tuple), str(next_board_tuple))
            dot.edge(str(board), str(next_board_tuple), label=f"{move}: {q_value:.2f}", color=color)
    return dot


In [ ]:
paths = brute_force(100, 10, verbose=False)
paths


In [ ]:
q_dict = build_q_dict(paths)
q_dict_graph = visualize_q_dict(q_dict)
q_dict_graph